In [16]:
# Load the annotation file
anno_path = '/workspace/work/O2ONet/data/annotations_minus_unavailable_yt_vids.pkl'

import pickle as pkl

f = open(anno_path, 'rb')
annotations = pkl.load(f)
f.close()

In [17]:
gif_folder = '/workspace/data/data_folder/o2o/gifs_11'
annotation = annotations[0]

# Getting details to load the GIF
yt_id = annotation['metadata']['yt_id']
frame_index = annotation['metadata']['frame no.']

temp = int(int(gif_folder.split('_')[-1])/2)
window_size = temp

# Loading the gif    
# getting the file location
filename = yt_id + '_' + str(frame_index) + '_' + str(window_size) + '.gif'
import os
file_location = os.path.join(gif_folder, filename)
import cv2

# getting the frames
vid = cv2.VideoCapture(file_location)
frames = []
frame_count = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
for i in range(frame_count):
    success, frame = vid.read()
    # frame = cv2.resize(frame, )
    frames.append(frame)
central_frame = frames[window_size]

Using CVPR22 Code

In [18]:
import pandas as pd
import numpy as np
import cv2
import torch
import torchvision
from torchvision.ops import MultiScaleRoIAlign
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torch.utils.data import DataLoader, Dataset
from torch import nn
import torchvision.transforms as transforms
import random
import os
from PIL import Image
import pytorchvideo.models as models
import torch.nn.functional as F

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
from collections import OrderedDict

In [19]:
from torch import nn
class FeatureExtractor(nn.Module):

    def __init__(self, submodule, layer):
        super(FeatureExtractor, self).__init__()
        self.pretrain_model = submodule
        self.layer = layer
        
        self.layer_list = list(self.pretrain_model._modules['blocks']._modules.keys())
        print(list(self.pretrain_model._modules['blocks']._modules))
        output_layer = self.layer_list[self.layer]  # just change the number of the layer to get the output

        self.children_list = []
        for (name, comp_layer) in self.pretrain_model._modules['blocks'].named_children():
            self.children_list.append(comp_layer)
            if name == output_layer:
                break
        #print(self.children_list)
        self.feature_extrac_net = nn.Sequential(*self.children_list)
        self.pretrain_model = None

    def forward(self, image):
        feature = self.feature_extrac_net(image)
        return feature


In [20]:
import json
import urllib
from pytorchvideo.data.encoded_video import EncodedVideo

from torchvision.transforms import Compose, Lambda
from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,
)

from pytorchvideo.transforms import (
    ApplyTransformToKey,
    ShortSideScale,
    UniformTemporalSubsample
)
side_size = 256
# mean = [0.45, 0.45, 0.45]
# std = [0.225, 0.225, 0.225]

mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

In [52]:
import imageio
import glob
import numpy as np

def read_gif(gif_path):
    """read gif and return dictionary as key ['video'] and value the tensor of size(CxTxHXW)"""
    video = EncodedVideo.from_path(gif_path)
    print(type(video))
    video = video.get_clip(0, 5)
    #print(video['video'].shape)
    return video
gif_location = '/workspace/data/data_folder/o2o/gifs_11/_1_dgZ4-Ldw_1262_5.gif'
vid = read_gif(gif_location)

<class 'pytorchvideo.data.encoded_video_pyav.EncodedVideoPyAV'>


In [53]:
transform =  ApplyTransformToKey(
    key="video",
    transform=Compose(
        [
            UniformTemporalSubsample(11),
            Lambda(lambda x: x/255.0),
            NormalizeVideo(mean, std)    
        ]
    ),
)

# del(model)
device = torch.device('cuda:2') if torch.cuda.is_available() else torch.device('cpu')
import pytorchvideo.models as models
model_name = "i3d_r50"
model = torch.hub.load("facebookresearch/pytorchvideo:main", model=model_name, pretrained=True)
model = model.to(device)
i3d_feature_net = FeatureExtractor(model, 4)

vid2 = transform(vid)
print(vid2["video"].shape)
vid2 = vid2["video"]
vid2 = vid2.unsqueeze(0).to(device)
feature = i3d_feature_net(vid2)
print(feature.shape, vid2.shape)

Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main


['0', '1', '2', '3', '4', '5', '6']
torch.Size([3, 11, 720, 1280])
torch.Size([1, 1024, 5, 45, 80]) torch.Size([1, 3, 11, 720, 1280])


In [37]:
# Doesn't need correction
def roi_align(feature_map, boxes):
    
    pooler = torchvision.ops.RoIAlign(output_size=(1, 1), spatial_scale = 1.0, sampling_ratio=1)
    boxes_list = [boxes]
    output = pooler(feature_map, boxes_list)

    return output

In [65]:
import torch
from torchvision.ops import RoIAlign

# output_size = (3, 3)
# spatial_scale = 1 / 4
# sampling_ratio = 2  

output_size = (1,1)
spatial_scale = 1
sampling_ratio = 1


x = torch.FloatTensor([[
    [[1,  2,  3,  4,  5,  6 ],
     [7,  8,  9,  10, 11, 12],
     [13, 14, 15, 16, 17, 18],
     [19, 20, 21, 22, 23, 24],
     [25, 26, 27, 28, 29, 30],
     [31, 32, 33, 34, 35, 36],
     [37, 38, 39, 40, 41, 42]]
]])

# rois = torch.tensor([
#     [0, -2.0, -2.0, 22.0, 22.0],
# ])

rois = torch.tensor([
    [0, 0.5, -0.5, 1.5, 0.5],
], dtype=torch.float)



a = RoIAlign(output_size, spatial_scale=spatial_scale, sampling_ratio=sampling_ratio)
ya = a(x, rois)
print(ya)

tensor([[[[2.]]]])


In [80]:
transform = Compose(
        [
            UniformTemporalSubsample(3),
        ]
    )

# c,t,h,w
x = torch.tensor([[[1],[1]], [[2],[2]], [[3],[3]], [[4],[4]], [[5],[5]], [[6],[6]], [[7],[7]], [[8],[8]], [[9],[9]], [[10],[10]], [[11],[11]] ])
print(transform(x))

ValueError: expected sequence of length 1 at dim 2 (got 0)

In [73]:
a = torch.randn(1,7)
print(a, a[:,0:4:2])

tensor([[-1.6512,  0.3391, -0.3355, -0.4076,  0.0512,  1.7046,  0.8876]]) tensor([[-1.6512, -0.3355]])


In [47]:
# Carefully look at the documentation to build the code for roi_feature extraction. Anyway to verify??
temp_box = torch.tensor([[2,3,4,5], [2,3,4,5], [2,3,4,5]], dtype=torch.float, device=device)
r = roi_align(feature[:,:,0,:,:], boxes=temp_box)
print(r.shape)

torch.Size([3, 1024, 1, 1])


Using the GitHub code

In [1]:
import os

from py import process
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
import sys
sys.path.append('/workspace/work/interactions_research/preproc/pytorch-i3d')
import argparse

import torch
from torch.autograd import Variable

from torchvision import datasets, transforms
import videotransforms

import numpy as np

from pytorch_i3d import InceptionI3d



import torch
import pynvml
def get_memory_free_MiB(gpu_index):
    pynvml.nvmlInit()
    handle = pynvml.nvmlDeviceGetHandleByIndex(int(gpu_index))
    mem_info = pynvml.nvmlDeviceGetMemoryInfo(handle)
    return mem_info.free // 1024 ** 2


test_transforms = transforms.Compose([videotransforms.CenterCrop(224)])
i3d_model_loc = '/workspace/work/interactions_research/preproc/pytorch-i3d/models/rgb_charades.pt'

i3d = InceptionI3d(400, in_channels=3)
i3d.replace_logits(157)
i3d.load_state_dict(torch.load(i3d_model_loc))
i3d.cuda()

i3d.train(False)  # Set model to evaluate mode
i3d.requires_grad_(False)

InceptionI3d(
  (avg_pool): AvgPool3d(kernel_size=[2, 7, 7], stride=(1, 1, 1), padding=0)
  (avg_pool_2d): AvgPool2d(kernel_size=[7, 7], stride=[7, 7], padding=0)
  (dropout): Dropout(p=0.5, inplace=False)
  (logits): Unit3D(
    (conv3d): Conv3d(1024, 157, kernel_size=(1, 1, 1), stride=(1, 1, 1))
  )
  (Conv3d_1a_7x7): Unit3D(
    (conv3d): Conv3d(3, 64, kernel_size=(7, 7, 7), stride=(2, 2, 2), bias=False)
    (bn): BatchNorm3d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
  )
  (MaxPool3d_2a_3x3): MaxPool3dSamePadding(kernel_size=[1, 3, 3], stride=(1, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (Conv3d_2b_1x1): Unit3D(
    (conv3d): Conv3d(64, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
    (bn): BatchNorm3d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
  )
  (Conv3d_2c_3x3): Unit3D(
    (conv3d): Conv3d(64, 192, kernel_size=(3, 3, 3), stride=(1, 1, 1), bias=False)
    (bn): BatchNorm3d(192, eps=0.001, momentum=0.01, aff

In [7]:
crop_size = 224
short_side_size = 256
transform_fn = transforms.Compose([transforms.Resize(short_side_size),
                                         transforms.CenterCrop(size=(crop_size, crop_size)),
                                         transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

In [6]:
f = torch.from_numpy(frames[0])

In [8]:
transform_fn(f)

KeyboardInterrupt: 

In [ ]:
i3d.extract_features(frames)

In [ ]:
input_segment_tensor = torch.from_numpy(processing_segment)
input_segment_tensor = test_transforms(input_segment_tensor[0])
input_segment_tensor = input_segment_tensor.unsqueeze(0)
transformed_segment = input_segment_tensor.cpu().detach().numpy()


In [ ]:
import numpy as np
import decord
import torch

from gluoncv.torch.utils.model_utils import download
from gluoncv.torch.data.transforms.videotransforms import video_transforms, volume_transforms
from gluoncv.torch.engine.config import get_cfg_defaults
from gluoncv.torch.model_zoo import get_model

In [ ]:
url = 'https://github.com/bryanyzhu/tiny-ucf101/raw/master/abseiling_k400.mp4'
video_fname = download(url)
vr = decord.VideoReader(video_fname)
frame_id_list = range(0, 64, 2)
video_data = vr.get_batch(frame_id_list).asnumpy()

In [ ]:
crop_size = 224
short_side_size = 256
transform_fn = video_transforms.Compose([video_transforms.Resize(short_side_size, interpolation='bilinear'),
                                         video_transforms.CenterCrop(size=(crop_size, crop_size)),
                                         volume_transforms.ClipToTensor(),
                                         video_transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])


clip_input = transform_fn(video_data)
print('Video data is downloaded and preprocessed.')

In [ ]:
config_file = '../../../scripts/action-recognition/configuration/i3d_resnet50_v1_kinetics400.yaml'
cfg = get_cfg_defaults()
cfg.merge_from_file(config_file)
model = get_model(cfg)
model.eval()
print('%s model is successfully loaded.' % cfg.CONFIG.MODEL.NAME)

In [ ]:
with torch.no_grad():
    pred = model(torch.unsqueeze(clip_input, dim=0)).numpy()
print('The input video clip is classified to be class %d' % (np.argmax(pred)))